In [1]:
import sys
print(sys.path)

['/Users/michelleng/Documents/Biobank/notebooks', '/private/var/folders/fl/2_tlvd8d3_z8t_8xgf3k_qvr0000gn/T/c90bcadc-c741-4580-89e8-c08127d12845', '/Users/michelleng/.vscode/extensions/ms-python.python-2020.5.78807/pythonFiles', '/Users/michelleng/.vscode/extensions/ms-python.python-2020.5.78807/pythonFiles/lib/python', '/Users/michelleng/opt/anaconda3/lib/python37.zip', '/Users/michelleng/opt/anaconda3/lib/python3.7', '/Users/michelleng/opt/anaconda3/lib/python3.7/lib-dynload', '', '/Users/michelleng/opt/anaconda3/lib/python3.7/site-packages', '/Users/michelleng/opt/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/michelleng/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/michelleng/.ipython']


# Hospitalisation events

# Packages

In [65]:
import pandas as pd 
import numpy as np 
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pickle
%matplotlib inline

# Dataset

In [17]:
opcs3 = [0,] + list(range(7369,7395))

In [18]:
opcs4 = [0,] + list(range(6622, 6671)) + list(range(7401, 7449))

In [19]:
death = [0,  6579, 6580, 6581, 6582]

In [20]:
# secondary care
opcs3_df = pd.read_csv('../primary_care/ukb41199.csv', usecols = opcs3)

In [21]:
opcs4_df = pd.read_csv('../primary_care/ukb41199.csv', usecols = opcs4)

In [22]:
death_df = pd.read_csv('../primary_care/ukb41199.csv', usecols = death)

In [23]:
#primary care
pri_cli = pd.read_csv('../primary_care/gp_clinical.txt', sep = '\t', encoding='ISO-8859-1')

In [24]:
#admission code (pri)
adm_codes = pd.read_excel('../primary_care/admission.xlsx', sheet_name = "admission")

In [25]:
#death code (death)
death_codes = pd.read_excel('../primary_care/admission.xlsx', sheet_name = "death")

In [56]:
opcs3_code = pd.read_csv('../primary_care/opcs3.tsv', sep = '\t')

# Event coding

In [26]:
event = {
    'hospitalAdm' : {
        'read2': list(adm_codes[adm_codes['Condition']== 'Related Admission']['READV2_CODE']),
        'read3': list(adm_codes[adm_codes['Condition']== 'Related Admission']['READV3_CODE'])
    },
    'death' : {
        'read2': list(death_codes['READV2_CODE'].values),
        'read3': list(death_codes['READV3_CODE'].values)
    }, 
    'all' : {
        'read2': (list(adm_codes[adm_codes['Condition']== 'Related Admission']['READV2_CODE'])+list(death_codes['READV2_CODE'].values)),
        'read3': (list(adm_codes[adm_codes['Condition']== 'Related Admission']['READV3_CODE'])+list(death_codes['READV3_CODE'].values))
    }
}

In [74]:
#isolating codes from primary care
hospitalised = pri_cli[pri_cli['read_2'].isin(event['hospitalAdm']['read2'])| pri_cli['read_3'].isin(event['hospitalAdm']['read3'])]

In [30]:
opcs3_col = list(opcs3_df.columns)
opcs3_df.columns

Index(['eid', '41256-0.0', '41256-0.1', '41256-0.2', '41256-0.3', '41256-0.4',
       '41256-0.5', '41256-0.6', '41256-0.7', '41256-0.8', '41256-0.9',
       '41256-0.10', '41256-0.11', '41256-0.12', '41257-0.0', '41257-0.1',
       '41257-0.2', '41257-0.3', '41257-0.4', '41257-0.5', '41257-0.6',
       '41257-0.7', '41257-0.8', '41257-0.9', '41257-0.10', '41257-0.11',
       '41257-0.12'],
      dtype='object')

In [32]:
opcs3_col = [x.replace('41256', 'opcs3') for x in opcs3_col]

In [34]:
opcs3_col = [x.replace('41257', 'opcs3date') for x in opcs3_col]

In [36]:
opcs3_df.columns = opcs3_col

In [37]:
opcs4_col = list(opcs4_df.columns)

In [40]:
opcs4_col = [x.replace('41200', 'opcs4') for x in opcs4_col]

In [41]:
opcs4_col = [x.replace('41260', 'opcs4date') for x in opcs4_col]

In [42]:
opcs4_df.columns = opcs4_col

In [46]:
death_df.columns = ['eid', 'dod-0', 'dod-1', 'causedeath-0', 'causedeath-1']

In [52]:
opcs4_df[['eid', 'opcs4-0.0', 'opcs4date-0.0']]

,eid,opcs4-0.0,opcs4date-0.0
0,1000019,NaN,NaN
1,1000022,G451,2013-12-28
2,1000035,C792,2014-12-09
3,1000046,Q075,1997-09-01
4,1000054,H513,2014-02-25
...,...,...,...
502500,6025018,F094,1997-04-15
502501,6025023,D135,2014-05-28
502502,6025031,H206,2011-03-24
502503,6025040,G437,2007-11-20


In [55]:
hospitalised

,eid,data_provider,event_dt,read_2,read_3,value1,value2,value3
10522,1000421,3,18/08/2008,NaN,8H21.,NaN,NaN,NaN
14747,1000568,3,30/11/2009,NaN,8H2..,NaN,NaN,NaN
14773,1000568,3,19/05/2010,NaN,8H2..,NaN,NaN,NaN
23822,1000713,3,29/04/2015,NaN,8H2..,NaN,NaN,NaN
27427,1000965,3,14/03/2004,NaN,8H2..,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
123600594,6022686,3,19/12/2010,NaN,8H2..,NaN,NaN,NaN
123602329,6022784,3,23/11/2004,NaN,8H2..,NaN,NaN,NaN
123603077,6022795,2,10/01/2015,8H2..,NaN,NaN,NaN,NaN
123606749,6022972,3,22/07/2014,NaN,XM01Y,NaN,NaN,NaN


In [64]:
opcs3_code.tail(50)

,coding,meaning,node_id,parent_id,selectable
1852,969,969 Other non-operative procedures,1869,15,Y
1853,9691,969.1 Other non-operative procedures : hypotherma,1870,15,Y
1854,9693,969.3 Other non-operative procedures : oxygen ...,1871,15,Y
1855,9694,969.4 Other non-operative procedures : electro...,1872,15,Y
1856,9695,969.5 Other non-operative procedures : other s...,1873,15,Y
1857,9696,969.6 Other non-operative procedures : oral ox...,1874,15,Y
1858,970,970 Local application,1875,15,Y
1859,971,971 Local injection,1876,15,Y
1860,972,"972 Nerve block, anaesthetic",1877,15,Y
1861,973,973 Spinal anaesthesia,1878,15,Y


In [62]:
opcs3_df['opcs3-0.0'].unique()

array([      nan, 7.042e+03, 9.140e+02, 7.816e+03, 7.040e+02, 1.632e+03,
       7.130e+02, 5.400e+02, 4.190e+02, 2.130e+02, 3.891e+03, 7.090e+02,
       2.310e+02, 6.512e+03, 3.811e+03, 9.130e+02, 4.310e+02, 4.011e+03,
       6.870e+02, 7.051e+03, 6.961e+03, 3.870e+02, 2.201e+03, 1.931e+03,
       5.641e+03, 6.330e+02, 8.931e+03, 8.480e+02, 9.120e+02, 1.942e+03,
       9.280e+02, 8.940e+02, 2.200e+01, 2.350e+02, 7.420e+02, 4.410e+02,
       7.421e+03, 4.802e+03, 6.080e+02, 1.952e+03, 2.510e+02, 2.440e+02,
       8.090e+02, 9.030e+02, 4.440e+02, 7.061e+03, 2.624e+03, 6.960e+02,
       9.160e+02, 7.050e+02, 7.001e+03, 9.790e+02, 7.100e+01, 7.002e+03,
       6.910e+02, 6.860e+02, 7.825e+03, 4.680e+02, 4.811e+03, 2.342e+03,
       8.910e+02, 6.270e+02, 7.052e+03, 9.940e+02, 2.194e+03, 5.220e+02,
       7.350e+02, 4.023e+03, 2.196e+03, 8.100e+02, 2.260e+02, 1.820e+02,
       3.390e+02, 7.391e+03, 2.562e+03, 2.300e+01, 8.281e+03, 8.930e+02,
       8.010e+02, 9.624e+03, 1.000e+00, 2.520e+02, 

In [67]:
records = pd.read_pickle('../primary_care/records.pkl')

In [80]:
hospitalised['datetime'] = [ datetime.strptime(x, '%d/%m/%Y') if type(x) == str else np.nan for x in hospitalised['event_dt']]

In [81]:
hospital_rec = hospitalised.groupby('eid').agg(list).reset_index()

In [82]:
hospital_rec

,eid,data_provider,event_dt,read_2,read_3,value1,value2,value3,datetime
0,1000421,[3],[18/08/2008],[nan],[8H21.],[nan],[nan],[nan],[2008-08-18 00:00:00]
1,1000568,"[3, 3]","[30/11/2009, 19/05/2010]","[nan, nan]","[8H2.., 8H2..]","[nan, nan]","[nan, nan]","[nan, nan]","[2009-11-30 00:00:00, 2010-05-19 00:00:00]"
2,1000713,[3],[29/04/2015],[nan],[8H2..],[nan],[nan],[nan],[2015-04-29 00:00:00]
3,1000965,"[3, 3]","[14/03/2004, 06/12/2015]","[nan, nan]","[8H2.., 8H2..]","[nan, nan]","[nan, nan]","[nan, nan]","[2004-03-14 00:00:00, 2015-12-06 00:00:00]"
4,1002460,"[3, 3, 3, 3]","[25/11/2014, 28/11/2014, 28/11/2014, 03/01/2016]","[nan, nan, nan, nan]","[8H2.., 8H2.., 8H2.., 8H2..]","[nan, nan, nan, nan]","[nan, nan, nan, nan]","[nan, nan, nan, nan]","[2014-11-25 00:00:00, 2014-11-28 00:00:00, 201..."
...,...,...,...,...,...,...,...,...,...
17072,6022422,[3],[02/06/2014],[nan],[8H2..],[nan],[nan],[nan],[2014-06-02 00:00:00]
17073,6022686,[3],[19/12/2010],[nan],[8H2..],[nan],[nan],[nan],[2010-12-19 00:00:00]
17074,6022784,[3],[23/11/2004],[nan],[8H2..],[nan],[nan],[nan],[2004-11-23 00:00:00]
17075,6022795,[2],[10/01/2015],[8H2..],[nan],[nan],[nan],[nan],[2015-01-10 00:00:00]


In [87]:
records

,eid,stroke,ha,date_diag,data_provider,event_dt,read_2,read_3,value1,value2,...,34-0.0,uniq_record_diag,sort_record,first_record_d,first_record_diag,first_stroke_after_d,yr_birth,noRecords,discrepancy,event_int
0,1000063,True,False,"[[09/01/2003, Stroke]]",NaN,NaN,NaN,NaN,NaN,NaN,...,1939.0,"[[09/01/2003, Stroke]]","[[2003-01-09 00:00:00, Stroke]]",2003-01-09,Stroke,NaT,1939-01-01,1,False,NaN
1,1000291,False,True,"[[23/07/2003, MI]]",NaN,NaN,NaN,NaN,NaN,NaN,...,1948.0,"[[23/07/2003, MI]]","[[2003-07-23 00:00:00, MI]]",2003-07-23,MI,NaT,1948-01-01,1,False,NaN
2,1000584,False,True,"[[05/11/1996, MI]]",NaN,NaN,NaN,NaN,NaN,NaN,...,1951.0,"[[05/11/1996, MI]]","[[1996-11-05 00:00:00, MI]]",1996-11-05,MI,NaT,1951-01-01,1,False,NaN
3,1001104,False,True,[[nan]],NaN,NaN,NaN,NaN,NaN,NaN,...,1938.0,[[nan]],[],NaT,NaN,NaT,1938-01-01,0,False,NaN
4,1001162,True,False,[[nan]],NaN,NaN,NaN,NaN,NaN,NaN,...,1947.0,[[nan]],[],NaT,NaN,NaT,1947-01-01,0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30521,6021699,NaN,NaN,[],"[3, 3]","[19/01/2012, 07/02/2012]","[nan, nan]","[XaINF, XaINF]","[nan, nan]","[nan, nan]",...,1949.0,"[[19/01/2012, MI], [07/02/2012, MI]]","[[2012-01-19 00:00:00, MI], [2012-02-07 00:00:...",2012-01-19,MI,NaT,1949-01-01,2,False,NaN
30522,6021968,NaN,NaN,[],[4],[02/06/2015],[G65..],[nan],[nan],[nan],...,1941.0,"[[02/06/2015, Stroke]]","[[2015-06-02 00:00:00, Stroke]]",2015-06-02,Stroke,NaT,1941-01-01,1,False,NaN
30523,6022686,NaN,NaN,[],"[3, 3, 3, 3, 3, 3]","[17/01/2011, 14/02/2011, 14/03/2011, 13/04/201...","[nan, nan, nan, nan, nan, nan]","[XE2aB, XE2aB, XE2aB, XE2aB, XE2aB, XE2aB]","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]",...,1954.0,"[[11/05/2011, Stroke], [21/05/2012, Stroke], [...","[[2011-01-17 00:00:00, Stroke], [2011-02-14 00...",2011-01-17,Stroke,2011-02-14,1954-01-01,6,False,0.919937
30524,6022831,NaN,NaN,[],[3],[01/07/2005],[nan],[XE0VK],[nan],[nan],...,1939.0,"[[01/07/2005, Stroke]]","[[2005-07-01 00:00:00, Stroke]]",2005-07-01,Stroke,NaT,1939-01-01,1,False,NaN


In [125]:
hosp = []
hosp_delta = []

for i in range(len(records['sort_record'])):
    
    hosp_stat = [] # accumulating individual patient's list of hospitalisation

    # if eid has a record
    if (len(hospital_rec[hospital_rec ['eid'] == records.iloc[i]['eid']]) == 1) == True:
        # iterate through the individual patient's record
        for j in range(len(records.iloc[i]['sort_record'])):

            idx = records.iloc[i]['eid']
            a = records.iloc[i]['sort_record'][j][0]

            b = hospital_rec[hospital_rec['eid']==idx]['datetime'].values

            for k in range(len(b)):
                timediff = a - b[0][k]
                
                # if patient's date matches within 7 days of hospitalisation date
                if timediff.days < 7 & timediff.days > (-7)  : 
                    hosp_stat.append('H')
                else:
                    hosp_stat.append(np.nan)
        hosp.append(hosp_stat)
    else:
        hosp.append([np.nan])

In [127]:
records['hospitalisation'] = hosp

In [129]:
death

[0, 6579, 6580, 6581, 6582]

In [130]:
#isolating codes from primary care
dead = pri_cli[pri_cli['read_2'].isin(event['death']['read2'])| pri_cli['read_3'].isin(event['death']['read3'])]

In [133]:
dead_id = dead.groupby('eid').agg(list).reset_index()

In [134]:
dead_id['deathstat'] = 1

In [135]:
dead_id

,eid,data_provider,event_dt,read_2,read_3,value1,value2,value3,deathstat
0,1002572,"[3, 3]","[30/06/2015, 30/06/2015]","[nan, nan]","[XM01Y, XaCJB]","[nan, nan]","[nan, nan]","[nan, nan]",1
1,1004098,[3],[17/04/2014],[nan],[XM01Y],[nan],[nan],[nan],1
2,1006885,[3],[14/06/2016],[nan],[XM01Y],[nan],[nan],[nan],1
3,1007044,[3],[27/02/2015],[nan],[XM01Y],[nan],[nan],[nan],1
4,1008117,[3],[29/07/2013],[nan],[XM01Y],[nan],[nan],[nan],1
...,...,...,...,...,...,...,...,...,...
3252,6016484,[3],[06/08/2012],[nan],[XM01Y],[nan],[nan],[nan],1
3253,6017668,[3],[25/06/2015],[nan],[XM01Y],[nan],[nan],[nan],1
3254,6020339,[3],[03/04/2013],[nan],[XM01Y],[nan],[nan],[nan],1
3255,6021388,[3],[29/04/2010],[nan],[XM01Y],[nan],[nan],[nan],1


In [139]:
dead_summ = dead_id[['eid', 'event_dt', 'deathstat']]

In [149]:
dead_summ.columns = ['eid', 'deathdate', 'deathstat']

In [150]:
records2 = pd.merge(records, dead_summ, how= 'left', on = ['eid', 'eid'] )

In [151]:
records2

,eid,stroke,ha,date_diag,data_provider,event_dt,read_2,read_3,value1,value2,...,first_record_d,first_record_diag,first_stroke_after_d,yr_birth,noRecords,discrepancy,event_int,hospitalisation,deathdate,deathstat
0,1000063,True,False,"[[09/01/2003, Stroke]]",NaN,NaN,NaN,NaN,NaN,NaN,...,2003-01-09,Stroke,NaT,1939-01-01,1,False,NaN,[nan],NaN,NaN
1,1000291,False,True,"[[23/07/2003, MI]]",NaN,NaN,NaN,NaN,NaN,NaN,...,2003-07-23,MI,NaT,1948-01-01,1,False,NaN,[nan],NaN,NaN
2,1000584,False,True,"[[05/11/1996, MI]]",NaN,NaN,NaN,NaN,NaN,NaN,...,1996-11-05,MI,NaT,1951-01-01,1,False,NaN,[nan],NaN,NaN
3,1001104,False,True,[[nan]],NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaT,1938-01-01,0,False,NaN,[nan],NaN,NaN
4,1001162,True,False,[[nan]],NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaT,1947-01-01,0,False,NaN,[nan],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30521,6021699,NaN,NaN,[],"[3, 3]","[19/01/2012, 07/02/2012]","[nan, nan]","[XaINF, XaINF]","[nan, nan]","[nan, nan]",...,2012-01-19,MI,NaT,1949-01-01,2,False,NaN,[nan],NaN,NaN
30522,6021968,NaN,NaN,[],[4],[02/06/2015],[G65..],[nan],[nan],[nan],...,2015-06-02,Stroke,NaT,1941-01-01,1,False,NaN,[nan],NaN,NaN
30523,6022686,NaN,NaN,[],"[3, 3, 3, 3, 3, 3]","[17/01/2011, 14/02/2011, 14/03/2011, 13/04/201...","[nan, nan, nan, nan, nan, nan]","[XE2aB, XE2aB, XE2aB, XE2aB, XE2aB, XE2aB]","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]",...,2011-01-17,Stroke,2011-02-14,1954-01-01,6,False,0.919937,"[nan, nan, nan, nan, nan, nan]",NaN,NaN
30524,6022831,NaN,NaN,[],[3],[01/07/2005],[nan],[XE0VK],[nan],[nan],...,2005-07-01,Stroke,NaT,1939-01-01,1,False,NaN,[nan],NaN,NaN


In [152]:
records2['nohosp'] = [x.count('H') for x in records2['hospitalisation']]

In [161]:
records2

,eid,stroke,ha,date_diag,data_provider,event_dt,read_2,read_3,value1,value2,...,first_record_diag,first_stroke_after_d,yr_birth,noRecords,discrepancy,event_int,hospitalisation,deathdate,deathstat,nohosp
0,1000063,True,False,"[[09/01/2003, Stroke]]",NaN,NaN,NaN,NaN,NaN,NaN,...,Stroke,NaT,1939-01-01,1,False,NaN,[nan],NaN,NaN,0
1,1000291,False,True,"[[23/07/2003, MI]]",NaN,NaN,NaN,NaN,NaN,NaN,...,MI,NaT,1948-01-01,1,False,NaN,[nan],NaN,NaN,0
2,1000584,False,True,"[[05/11/1996, MI]]",NaN,NaN,NaN,NaN,NaN,NaN,...,MI,NaT,1951-01-01,1,False,NaN,[nan],NaN,NaN,0
3,1001104,False,True,[[nan]],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,1938-01-01,0,False,NaN,[nan],NaN,NaN,0
4,1001162,True,False,[[nan]],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,1947-01-01,0,False,NaN,[nan],NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30521,6021699,NaN,NaN,[],"[3, 3]","[19/01/2012, 07/02/2012]","[nan, nan]","[XaINF, XaINF]","[nan, nan]","[nan, nan]",...,MI,NaT,1949-01-01,2,False,NaN,[nan],NaN,NaN,0
30522,6021968,NaN,NaN,[],[4],[02/06/2015],[G65..],[nan],[nan],[nan],...,Stroke,NaT,1941-01-01,1,False,NaN,[nan],NaN,NaN,0
30523,6022686,NaN,NaN,[],"[3, 3, 3, 3, 3, 3]","[17/01/2011, 14/02/2011, 14/03/2011, 13/04/201...","[nan, nan, nan, nan, nan, nan]","[XE2aB, XE2aB, XE2aB, XE2aB, XE2aB, XE2aB]","[nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan]",...,Stroke,2011-02-14,1954-01-01,6,False,0.919937,"[nan, nan, nan, nan, nan, nan]",NaN,NaN,0
30524,6022831,NaN,NaN,[],[3],[01/07/2005],[nan],[XE0VK],[nan],[nan],...,Stroke,NaT,1939-01-01,1,False,NaN,[nan],NaN,NaN,0


In [172]:
records2['deathstat'].isnull().value_counts()

True     29709
False      817
Name: deathstat, dtype: int64

In [155]:
records2.to_pickle('../primary_care/records2.pkl')